# Universal Plasmid Maker — Example Run

This notebook demonstrates a complete example run of the **Universal Plasmid Maker** pipeline.

It shows how:
- an insert sequence (`Input.Fa`)
- a design specification (`Design.txt`)

are combined to generate a fully assembled plasmid sequence.


In [1]:
from Bio import SeqIO
from plasmid_builder.pipeline import run_pipeline

print("Pipeline imported successfully")


Pipeline imported successfully


In [2]:
# Create a small example insert sequence
insert_fasta = "example_input.fa"

with open(insert_fasta, "w") as f:
    f.write(
        ">example_insert\n"
        "ATGCATGCATGCATGC\n"
    )

print(f"Created {insert_fasta}")

Created example_input.fa


In [3]:
design_file = "example_design.txt"

with open(design_file, "w") as f:
    f.write(
        "Multiple_Cloning_Site1, EcoRI\n"
        "Multiple_Cloning_Site2, GAATTC\n"
        "ATGCCCAGTACGTTAGC, Kanamycin\n"
    )

print(f"Created {design_file}")

Created example_design.txt


In [4]:
output_fasta = "example_output.fa"
output_genbank = "example_output.gb"

plasmid = run_pipeline(
    input_fasta=insert_fasta,
    design_file=design_file,
    output_fasta=output_fasta,
    output_genbank=output_genbank
)

print("Pipeline completed successfully")

Pipeline completed successfully


In [5]:
print(f"Plasmid length: {len(plasmid.seq)} bp")
print("\nFirst 200 bp of assembled plasmid:\n")
print(plasmid.seq[:200])

Plasmid length: 8729 bp

First 200 bp of assembled plasmid:

AACTGCACATTCGGGATATTTCTCTATATTCGCGCTTCATCAGAAAACTGAAGGAACCTCCATTGAATCGAACTAATATTTTTTTTGGTGAATCGCATTCTGACTGGTTGCCTGTCAGAGGCGGAGAATCTGGTGATTTTGTTTTTCGACGTGGTGACGGGCATGCCTTCGCGAAAATCGCACCTGCTTCCCGCCGCGGT


In [6]:
print("Annotated features:\n")

for feature in plasmid.features:
    start = int(feature.location.start)
    end = int(feature.location.end)
    ftype = feature.type
    qualifiers = dict(feature.qualifiers)

    print(f"{ftype}: {start}–{end}")
    for k, v in qualifiers.items():
        print(f"  {k}: {v}")
    print()

Annotated features:

CDS: 5889–6729
  gene: repA
  product: replication protein A
  protein_id: AAA26450.1

CDS: 4407–5378
  gene: repB
  product: replication protein B
  protein_id: AAA26447.1

CDS: 6715–7566
  gene: repC
  product: replication protein C
  protein_id: AAA26451.1

CDS: 8684–8701
  note: antibiotic resistance marker
  antibiotic: Kanamycin

misc_feature: 8701–8713
  note: Multiple Cloning Site

insert: 8713–8729
  note: Inserted DNA



In [7]:
print("Generated files:")
print(f"- {output_fasta}")
print(f"- {output_genbank}")

Generated files:
- example_output.fa
- example_output.gb
